LOGGING:
streamHandler = it will type all error message in the termal
file handler = it will do,in this file write all the logs, the selected file errors will be logged.

LIBRARIES USE:
1. ConfigBox : to acces variables in easy way 
    Ex: 

In [9]:
d = {"key": "value", "key1": "value1"}
d.key

AttributeError: 'dict' object has no attribute 'key'

In [7]:
from box import ConfigBox
d1 = ConfigBox({"key": "value", "key1": "value1"})
d1.key


'value'

Ensure lib : used to give specific data type, so when given datatype is not matched to the written code then it will through error.
EX:-

In [10]:
from ensure import ensure_annotations

In [11]:
def get_product1(x: int, y: int) -> int:
    return x*y

get_product1(x=2,y='hi')

'hihi'

In [13]:
@ensure_annotations
def get_product(x: int, y: int) -> int:
    return x*y

get_product(x=3,y='hi')

EnsureError: Argument y of type <class 'str'> to <function get_product at 0x0000026A5154AAE8> does not match annotation type <class 'int'>